## Classification
## Example: Predict survival on Titanic

In [73]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline

## Working with data

In [74]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [75]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [76]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [77]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


### We'll need test_pas_id for submission dataframe

In [78]:
test_pas_id = test['PassengerId']

### Make a list from train and test

In [79]:
full_data=[train, test]

In [80]:
type(full_data)

list

### Impute missing values

#### Embarked

In [81]:
train[train['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [82]:
train[(train['Fare']>79) & (train['Fare']<81) & (train['Pclass']==1)].groupby('Embarked').size()

Embarked
C    4
S    3
dtype: int64

In [83]:
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('C')

#### Fare

In [84]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

####  Age

In [85]:
# We have plenty of missing values in this feature. 
# Generate random numbers between (mean - std) and (mean + std). 

In [86]:
train['Age'].isnull().sum()

177

In [87]:
np.random.seed(0)
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset.loc[dataset['Age'].isnull(), 'Age'] = age_null_random_list 

In [88]:
train['Age'].isnull().sum()

0

### Data preprocessing

In [89]:
np.unique(full_data[0]['Sex'], return_counts = True)

(array(['female', 'male'], dtype=object), array([314, 577], dtype=int64))

In [90]:
  for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} )

In [92]:
np.unique(full_data[0]['Embarked'], return_counts = True)

(array(['C', 'Q', 'S'], dtype=object), array([170,  77, 644], dtype=int64))

In [93]:
for dataset in full_data:
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'C': 0, 'Q': 1, 'S': 2} ).astype(int)

In [94]:
train['Embarked'].dtype

dtype('int32')

In [95]:
np.unique(full_data[0]['Pclass'], return_counts = True)

(array([1, 2, 3], dtype=int64), array([216, 184, 491], dtype=int64))

In [96]:
factors_train = train[['Pclass','Embarked']]
factors_test = test[['Pclass','Embarked']]

In [97]:
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder(dtype = 'int32')
enc.fit(factors_train)

OneHotEncoder(categorical_features='all', dtype='int32',
       handle_unknown='error', n_values='auto', sparse=True)

In [98]:
train['Pclass'][:5,]

0    3
1    1
2    3
3    1
4    3
Name: Pclass, dtype: int64

In [99]:
tfactors_train = enc.transform(factors_train).toarray()
tfactors_train[:5,]

array([[0, 0, 1, 0, 0, 1],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1],
       [1, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 1]], dtype=int32)

In [100]:
tfactors_test = enc.transform(factors_test).toarray()
tfactors_test[:5,]

array([[0, 0, 1, 0, 1, 0],
       [0, 0, 1, 0, 0, 1],
       [0, 1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 1]], dtype=int32)

In [101]:
train_pclass_emb = pd.DataFrame(tfactors_train, columns = ('Pclass_1', 'Pclass_2', 'Pclass_3', 'Emb_C', 'Emb_Q', 'Emb_S'))

In [102]:
train_pclass_emb.head()

,Pclass_1,Pclass_2,Pclass_3,Emb_C,Emb_Q,Emb_S
0,0,0,1,0,0,1
1,1,0,0,1,0,0
2,0,0,1,0,0,1
3,1,0,0,0,0,1
4,0,0,1,0,0,1


In [103]:
test_pclass_emb = pd.DataFrame(tfactors_test, columns = ('Pclass_1', 'Pclass_2', 'Pclass_3', 'Emb_C', 'Emb_Q', 'Emb_S'))

In [104]:
test_pclass_emb.head()

,Pclass_1,Pclass_2,Pclass_3,Emb_C,Emb_Q,Emb_S
0,0,0,1,0,1,0
1,0,0,1,0,0,1
2,0,1,0,0,1,0
3,0,0,1,0,0,1
4,0,0,1,0,0,1


### Feature Selection

In [105]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Pclass', 'Embarked']
train = train.drop(drop_elements, axis = 1)
test = test.drop(drop_elements, axis = 1)

In [106]:
train = pd.concat([train,train_pclass_emb], axis=1)
test = pd.concat([test,test_pclass_emb], axis=1)

In [107]:
train.columns

Index(['Survived', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Emb_C', 'Emb_Q', 'Emb_S'],
      dtype='object')

In [108]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
Survived    891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Pclass_1    891 non-null int32
Pclass_2    891 non-null int32
Pclass_3    891 non-null int32
Emb_C       891 non-null int32
Emb_Q       891 non-null int32
Emb_S       891 non-null int32
dtypes: float64(2), int32(6), int64(4)
memory usage: 62.7 KB


In [109]:
trainv = train.values

In [110]:
trainv.shape

(891, 12)

In [111]:
type(trainv)

numpy.ndarray

In [112]:
trainv.dtype

dtype('float64')

In [113]:
X = trainv[0:, 1:]
y = trainv[0:, 0]

In [114]:
X.shape

(891, 11)

In [115]:
from sklearn.model_selection import train_test_split
# Split to train and test
# 75% and 25% by default
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=2)
print(Xtrain.shape, Xtest.shape)

(668, 11) (223, 11)


In [116]:
# Standardize features by removing the mean and scaling to unit variance
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
# http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Xtrain)
Xtrain = scaler.transform(Xtrain)
Xtest = scaler.transform(Xtest)

### Data prepared to predict for submission

In [117]:
test.columns

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass_1', 'Pclass_2',
       'Pclass_3', 'Emb_C', 'Emb_Q', 'Emb_S'],
      dtype='object')

In [118]:
Xnew = scaler.transform(test.values)
Xnew.shape

(418, 11)

## Modeling

In [119]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report

In [120]:
np.unique(ytrain, return_counts = True)

(array([0., 1.]), array([418, 250], dtype=int64))

In [121]:
np.unique(ytest, return_counts = True)

(array([0., 1.]), array([131,  92], dtype=int64))

## LogisticRegression

In [122]:
# http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
# http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [152]:
# Model
model_lgr = LogisticRegression(random_state = 1)
print(model_lgr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [153]:
# C - Inverse of regularization strength; must be a positive float.
# Smaller values specify stronger regularization.

### Fit the model

In [154]:
model_lgr.fit(Xtrain, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Model fit parameters

In [155]:
model_lgr.coef_

array([[-2.60280628, -0.03470115, -0.360325  , -0.0142617 ,  0.00373809,
         1.3430553 ,  0.65335511, -0.53658209,  0.6331887 ,  0.5465939 ,
         0.28004571]])

In [156]:
model_lgr.coef_.reshape(11,)

array([-2.60280628, -0.03470115, -0.360325  , -0.0142617 ,  0.00373809,
        1.3430553 ,  0.65335511, -0.53658209,  0.6331887 ,  0.5465939 ,
        0.28004571])

In [157]:
params = pd.Series(model_lgr.coef_.reshape(11,), index=train.columns[1:])
params

Sex        -2.602806
Age        -0.034701
SibSp      -0.360325
Parch      -0.014262
Fare        0.003738
Pclass_1    1.343055
Pclass_2    0.653355
Pclass_3   -0.536582
Emb_C       0.633189
Emb_Q       0.546594
Emb_S       0.280046
dtype: float64

In [158]:
model_lgr.intercept_

array([1.45982832])

### Model validation

In [159]:
# Predict on train

ypred_train = model_lgr.predict(Xtrain)
ypred_train_proba = model_lgr.predict_proba(Xtrain)

In [160]:
# Predict on test

ypred = model_lgr.predict(Xtest)
print(ypred[:10])

ypred_proba = model_lgr.predict_proba(Xtest)
print(ypred_proba[:5,:])

# ypred_proba[:,0] - probability for class zero (not survived), 
# ypred_proba[:,1] - probability for class one - survived

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[[0.70662748 0.29337252]
 [0.89072294 0.10927706]
 [0.19552074 0.80447926]
 [0.89659588 0.10340412]
 [0.69568354 0.30431646]]


#### Metrics: accuracy, confusion matrix, classification report, AUC
#### http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics

In [161]:
# We can check our classification accuracy by comparing 
# the true values of the test set to the predictions:

In [162]:
# Accuracy on train
accuracy_score(ytrain, ypred_train)

0.8174157303370787

In [163]:
# Accuracy on test
accuracy_score(ytest, ypred)

0.770949720670391

In [164]:
# Score for classification models is accuracy
model_lgr.score(Xtest, ytest)

0.770949720670391

In [165]:
# Accuracy doesn't tell us where we've gone wrong: 
# one nice way to do this is to use the confusion matrix

In [166]:
print(confusion_matrix(ytest, ypred))

[[91  9]
 [32 47]]


In [167]:
target_names = ['not survived', 'survived']
print(classification_report(ytest, ypred, target_names=target_names))

              precision    recall  f1-score   support

not survived       0.74      0.91      0.82       100
    survived       0.84      0.59      0.70        79

 avg / total       0.78      0.77      0.76       179



In [168]:
# AUC
# y_scores -  probability estimates of the positive class

print("AUC on traint =", roc_auc_score(ytrain, ypred_train_proba[:, 1]))
print("AUC on test =", roc_auc_score(ytest, ypred_proba[:, 1]))

AUC on traint = 0.8642992031298958
AUC on test = 0.8241772151898735


### K-fold Cross-Validation

In [169]:
# http://scikit-learn.org/stable/modules/cross_validation.html

from sklearn.model_selection import cross_val_score

In [170]:
lgr = LogisticRegression(random_state = 1)

scores = cross_val_score(lgr, Xtrain, ytrain, cv=5)
scores

array([0.84615385, 0.81118881, 0.81818182, 0.8028169 , 0.78014184])

In [171]:
print("Mean cv accuracy : %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Mean cv accuracy : 0.81 (+/- 0.04)


In [172]:
lgr.fit(Xtrain,ytrain)
print("Accuracy on train",lgr.score(Xtrain,ytrain))
print("Accuracy on test", lgr.score(Xtest, ytest))

Accuracy on train 0.8174157303370787
Accuracy on test 0.770949720670391


### Hyperparameters Grid Search

In [176]:
# http://scikit-learn.org/stable/modules/grid_search.html#grid-search

# GridSearchCV exhaustively considers all parameter combinations

from sklearn.model_selection import GridSearchCV

param_grid = {'C': [.001, .01, 1, 10],
              'penalty': ['l1', 'l2']}
lgr = LogisticRegression(random_state=1)
grid = GridSearchCV(lgr, param_grid, cv=5)

In [177]:
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 1, 10], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [178]:
# Mean cross-validated score of the best_estimator
grid.best_score_

0.8117977528089888

In [179]:
grid.best_params_

{'C': 1, 'penalty': 'l2'}

In [180]:
model = grid.best_estimator_

In [181]:
print(model)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [182]:
model.score(Xtest,ytest)

0.770949720670391

In [183]:
model.score(Xtrain,ytrain)

0.8174157303370787

### Save / load a model

In [184]:
from sklearn.externals import joblib
joblib.dump(model, 'model.pkl') 

['model.pkl']

In [185]:
model1 = joblib.load('model.pkl') 

In [186]:
print(model1)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [187]:
ypred = model1.predict(Xtest)
ypred[:10]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

## RandomForestClassifier

In [3]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# rfc = RandomForestClassifier(random_state = ...)

In [ ]:
# Fit the model


In [ ]:
# Model accuracy on train and on test data


In [ ]:
# Features importance
# pd.Series(rfc.feature_importances_, index=train.columns[1:])

In [ ]:
# Predict on test

In [ ]:
# Print classification report

#### Hyperparameters Grid Search

In [ ]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {'n_estimators': [5, 10, 15, 20],
#              'max_depth': [2, 3, 4, 5, 6, 7, 8]}
# grid = GridSearchCV(RandomForestClassifier(random_state = 2), param_grid, cv=5)

In [ ]:
# Fit grid


In [ ]:
# Mean cross-validated score of the best_estimator


In [ ]:
# Parameters (max_depth and n_estimators) of the best_estimator


In [ ]:
# Accuracy of the best_estimator on train and test

### GradientBoostingClassifier

In [166]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Continue on analogue with RandomForestClassifier

### XGBoost
#### http://xgboost.readthedocs.io/en/latest/python/python_intro.html

### Submission

In [54]:
# Generate Submission File 

# Use model with the best accuracy on test to predict on Xnew (ypred_Xnew should be int)

# Example: ypred_Xnew = model_lgr.predict(Xnew).astype(int)

# submission = pd.DataFrame({ 'PassengerId': test_pas_id,
#                             'Survived': ypred_Xnew })
# submission.to_csv("submission.csv", index=False)

1) Register on https://www.kaggle.com
2) Go to https://www.kaggle.com/c/titanic/submit
3) Submit your csv file and get the score (accuracy)